# Fine-Tune BERT With Sentence Transformers

### With NLI Softmax Loss

In [1]:
import datasets

snli = datasets.load_dataset('snli', split='train')
mnli = datasets.load_dataset('glue', 'mnli', split='train')
snli, mnli

(Dataset({
     features: ['premise', 'hypothesis', 'label'],
     num_rows: 550152
 }),
 Dataset({
     features: ['premise', 'hypothesis', 'label', 'idx'],
     num_rows: 392702
 }))

In [2]:
mnli = mnli.remove_columns(['idx'])
snli = snli.cast(mnli.features)

nli = datasets.concatenate_datasets([snli, mnli])

nli = nli.filter(
    lambda x: False if x['label'] == -1 else True
)

nli

Dataset({
    features: ['premise', 'hypothesis', 'label'],
    num_rows: 942069
})

### Getting Ready fwith Sentence Transformers

In [3]:
from tqdm.auto import tqdm
from sentence_transformers import InputExample

train_samples  =[]

for row in tqdm(nli):
    train_samples.append(InputExample(
        texts=[row['premise'], row['hypothesis']],
        label=row['label']
    ))

  0%|          | 0/942069 [00:00<?, ?it/s]

In [4]:
from torch.utils.data import DataLoader

batch_size = 16

loader = DataLoader(train_samples, shuffle=True, batch_size=batch_size)

### Training BERT Model

In [5]:
from sentence_transformers import models, SentenceTransformer

modelName = 'bert-base-uncased'

bert = models.Transformer(modelName)
pooler = models.Pooling(bert.get_word_embedding_dimension(),
                        pooling_mode_mean_tokens=True)

model = SentenceTransformer(modules=[bert, pooler])
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False})
)

In [6]:
from sentence_transformers.losses import SoftmaxLoss

loss = SoftmaxLoss(
    model=model,
    sentence_embedding_dimension=model.get_sentence_embedding_dimension(),
    num_labels=3
)

In [7]:
epochs = 1
warmup_steps = int(len(loader) * epochs * 0.1)

model.fit(
    train_objectives=[(loader, loss)],
    epochs=epochs,
    warmup_steps=warmup_steps,
    output_path='saved-models',
    show_progress_bar=True
)

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/58880 [00:00<?, ?it/s]

In [8]:
model.save_pretrained("saved-models")

AttributeError: 'SentenceTransformer' object has no attribute 'save_pretrained'